# Taller de Análisis Numérico Lineal: Integración Numérica
**Fecha:** 9 de enero de 2026

Este cuaderno contiene la solución rigurosa a los ejercicios planteados sobre métodos de integración numérica (Trapecio, Simpson, Romberg, Adaptativo) y análisis de errores para funciones suaves y singulares.

In [ ]:
# Importación de librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import quad

# Configuración de gráficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Librerías cargadas correctamente.")

## 1. Análisis Teórico de la Función $f(x)$

Sea la función:
$$ f(x) = e^{-x}\cos(5x), \quad x \in [0, 2] $$

### a) Justificación de suavidad ($C^{\infty}$)
La función $f(x)$ es el producto de dos funciones elementales:
1. $h(x) = e^{-x}$ (Exponencial, $C^{\infty}$ en $\mathbb{R}$).
2. $k(x) = \cos(5x)$ (Trigonométrica, $C^{\infty}$ en $\mathbb{R}$).

**Teorema:** El producto de funciones de clase $C^{\infty}$ es de clase $C^{\infty}$. Por tanto, $f(x)$ es infinitamente diferenciable en $[0, 2]$, sin singularidades ni discontinuidades.

### b) Existencia y Unicidad de la Integral
**Teorema:** Toda función continua en un intervalo cerrado y acotado $[a, b]$ es Riemann-integrable.
Dado que $f \in C^{\infty}([0,2]) \subset C^0([0,2])$, la integral $I_f = \int_0^2 f(x)dx$ existe y es única.

In [ ]:
# Definición de la función y parámetros
def f(x):
    return np.exp(-x) * np.cos(5*x)

a, b = 0, 2

# 2. Cálculo del valor de referencia (QUADPACK)
I_ref, error_est = quad(f, a, b, epsabs=1e-12, epsrel=1e-12)

print(f"Valor de referencia (I_ref): {I_ref:.15f}")
print(f"Error estimado por quad:     {error_est:.15e}")

In [ ]:
# 3. Implementación de las Reglas de Integración Compuestas

def composite_midpoint(func, a, b, n):
    """Regla del Punto Medio Compuesta"""
    h = (b - a) / n
    x_mid = np.linspace(a + h/2, b - h/2, n)
    return h * np.sum(func(x_mid))

def composite_trapezoidal(func, a, b, n):
    """Regla del Trapecio Compuesta"""
    x = np.linspace(a, b, n + 1)
    y = func(x)
    h = (b - a) / n
    return (h/2) * (y[0] + 2*np.sum(y[1:-1]) + y[-1])

def composite_simpson(func, a, b, n):
    """Regla de Simpson 1/3 Compuesta (n debe ser par)"""
    if n % 2 != 0:
        raise ValueError("n debe ser par para Simpson")
    x = np.linspace(a, b, n + 1)
    y = func(x)
    h = (b - a) / n
    return (h/3) * (y[0] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-2:2]) + y[-1])

# 4. y 5. Cálculos para n=4 y n=8
results = []
for n in [4, 8]:
    im = composite_midpoint(f, a, b, n)
    it = composite_trapezoidal(f, a, b, n)
    is_ = composite_simpson(f, a, b, n)
    
    results.append({
        "n": n,
        "Midpoint": im,
        "Trapezoid": it,
        "Simpson": is_,
        "Err_Mid": abs(I_ref - im),
        "Err_Trap": abs(I_ref - it),
        "Err_Simp": abs(I_ref - is_)
    })

df_results = pd.DataFrame(results)
display(df_results)

## 5. Análisis de Convergencia

El orden empírico de convergencia $p$ se estima mediante:
$$ p \approx \log_2\left(\frac{E(n_1)}{E(n_2)}\right) $$

* **Trapecio / Punto Medio:** Teóricamente $O(h^2) \implies p \approx 2$.
* **Simpson:** Teóricamente $O(h^4) \implies p \approx 4$.

In [ ]:
# Cálculo del orden de convergencia empírico
err_4 = df_results.iloc[0]
err_8 = df_results.iloc[1]

p_mid = np.log2(err_4["Err_Mid"] / err_8["Err_Mid"])
p_trap = np.log2(err_4["Err_Trap"] / err_8["Err_Trap"])
p_simp = np.log2(err_4["Err_Simp"] / err_8["Err_Simp"])

print(f"Orden de convergencia empírico (n=4 -> n=8):")
print(f"Punto Medio: {p_mid:.2f} (Teórico: 2)")
print(f"Trapecio:    {p_trap:.2f} (Teórico: 2)")
print(f"Simpson:     {p_simp:.2f} (Teórico: 4)")

In [ ]:
# 6. y 7. Cálculos para n=16 y Análisis Oscilatorio

n_16 = 16
val_simp_16 = composite_simpson(f, a, b, n_16)
val_trap_16 = composite_trapezoidal(f, a, b, n_16)

print(f"n=16 | Error Trapecio: {abs(I_ref - val_trap_16):.5e}")
print(f"n=16 | Error Simpson:  {abs(I_ref - val_simp_16):.5e}")

# Visualización de la función
x_plot = np.linspace(a, b, 200)
plt.figure(figsize=(10,4))
plt.plot(x_plot, f(x_plot), label=r"$f(x) = e^{-x}\cos(5x)$", color='blue')
plt.title("Naturaleza oscilatoria de la función")
plt.axhline(0, color='black', lw=0.5)
plt.legend()
plt.show()

### Análisis del Efecto Oscilatorio
La función tiene una frecuencia angular $\omega = 5$. El error en Simpson depende de $f^{(4)}(x)$, que escala con $\omega^4 = 625$. A pesar de esto, como $h$ disminuye, el factor $h^4$ domina y reduce el error rápidamente. Sin embargo, para $n$ muy pequeños (paso $h$ grande), la regla del trapecio puede "cortar" los ciclos de oscilación perdiendo información significativa, mientras que Simpson se adapta mejor a la curvatura.

In [ ]:
# 8. Tabla de Romberg (Extrapolación de Richardson)

# Matriz R para almacenar resultados
R = np.zeros((3, 3))

# Columna 0: Trapecio para n=1, 2, 4
R[0, 0] = composite_trapezoidal(f, a, b, 1)
R[1, 0] = composite_trapezoidal(f, a, b, 2)
R[2, 0] = composite_trapezoidal(f, a, b, 4)

# Columna 1: Extrapolación O(h^4) (Similar a Simpson)
for i in range(1, 3):
    R[i, 1] = (4**1 * R[i, 0] - R[i-1, 0]) / (4**1 - 1)

# Columna 2: Extrapolación O(h^6) (Similar a Boole)
for i in range(2, 3):
    R[i, 2] = (4**2 * R[i, 1] - R[i-1, 1]) / (4**2 - 1)

print("Tabla de Romberg:")
print(R)
print(f"\nAproximación R_2,2: {R[2,2]:.15f}")
print(f"Error R_2,2:        {abs(I_ref - R[2,2]):.5e}")

## 9. Análisis de Romberg
**a) Hipótesis:** La extrapolación funciona porque $f \in C^{\infty}$, lo que permite expandir el error de la regla del trapecio en una serie de potencias pares de $h$ (Euler-Maclaurin).

**b) Costo:** Romberg reutiliza evaluaciones anteriores. Para obtener $R_{2,2}$ solo se requirió calcular hasta $n=4$ trapecios, logrando una precisión superior a Simpson con el mismo número de evaluaciones.

In [ ]:
# 10. Integración Adaptativa (Simpson)

x_nodes = [] # Para visualizar distribución de puntos

def simpson_step(f, a, b):
    c = (a + b) / 2
    h = (b - a) / 2
    return (h/3) * (f(a) + 4*f(c) + f(b))

def adaptive_simpson_recursive(f, a, b, tol, whole_simpson):
    c = (a + b) / 2
    left_simpson = simpson_step(f, a, c)
    right_simpson = simpson_step(f, c, b)
    
    # Estimación de error
    error = (left_simpson + right_simpson - whole_simpson) / 15
    
    if abs(error) <= tol:
        x_nodes.append(a); x_nodes.append(c); x_nodes.append(b)
        return left_simpson + right_simpson + error
    else:
        return (adaptive_simpson_recursive(f, a, c, tol/2, left_simpson) + 
                adaptive_simpson_recursive(f, c, b, tol/2, right_simpson))

def solve_adaptive(f, a, b, tol):
    x_nodes.clear()
    initial_simp = simpson_step(f, a, b)
    return adaptive_simpson_recursive(f, a, b, tol, initial_simp)

tol = 1e-6
I_adapt = solve_adaptive(f, 0, 2, tol)
unique_nodes = np.unique(x_nodes)

print(f"Resultado Adaptativo: {I_adapt:.15f}")
print(f"Nodos únicos usados:  {len(unique_nodes)}")
print(f"Error real:           {abs(I_ref - I_adapt):.5e}")

# Visualización del refinamiento
plt.figure(figsize=(10, 2))
plt.scatter(unique_nodes, np.zeros_like(unique_nodes), alpha=0.5, s=10, color='red')
plt.title("Distribución de nodos (Refinamiento Adaptativo)")
plt.xlabel("x")
plt.yticks([])
plt.show()

In [ ]:
# 13. Análisis de función singular g(x)

def g(x):
    # Evitamos log(0) numérico, sabiendo que el límite es 0
    val = np.sqrt(x) * np.log(x + 1)
    return np.where(x==0, 0, val)

# Referencia
I_g_ref, _ = quad(g, 0, 1)

# Comparación
n_g = 32
trap_g = composite_trapezoidal(g, 0, 1, n_g)
simp_g = composite_simpson(g, 0, 1, n_g)
adapt_g = solve_adaptive(g, 0, 1, 1e-6)

print(f"Referencia g(x): {I_g_ref:.10f}")
print(f"Trapecio (n={n_g}): {trap_g:.10f} | Error: {abs(I_g_ref - trap_g):.5e}")
print(f"Simpson (n={n_g}):  {simp_g:.10f} | Error: {abs(I_g_ref - simp_g):.5e}")
print(f"Adaptativo:      {adapt_g:.10f} | Error: {abs(I_g_ref - adapt_g):.5e}")

# Gráfica de g(x)
plt.figure(figsize=(10,4))
x_vals = np.linspace(0, 1, 500)
plt.plot(x_vals, g(x_vals))
plt.title(r"$g(x) = \sqrt{x}\ln(x+1)$")
plt.show()

### Análisis de la Singularidad de $g(x)$
**a) No pertenencia a $C^2$:**
Cerca de 0, $g(x) \approx x^{3/2}$. Su segunda derivada es $g''(x) \approx \frac{3}{4}x^{-1/2}$.
Cuando $x \to 0$, $g''(x) \to \infty$. La función no tiene segunda derivada acotada en $[0,1]$, lo que viola las hipótesis de error para Trapecio y Simpson.

**b) Consecuencia:**
Simpson pierde su orden de convergencia $O(h^4)$ y se vuelve ineficiente. El método adaptativo, sin embargo, detecta el error masivo cerca de 0 y refina agresivamente en esa zona, manteniendo la precisión global.

## 14. Conclusiones Finales

1. **Selección de Método:**
    * *Suaves:* Romberg o Gauss (Alta eficiencia).
    * *Oscilatorias:* Adaptativos de alto orden o Filon.
    * *Singulares:* Adaptativos o cambio de variable.

2. **Universalidad:**
    No existe un método universal. La eficiencia depende estrictamente de la regularidad (suavidad) de la función. Los métodos robustos (adaptativos) son más seguros pero computacionalmente más costosos si no son necesarios.